<a href="https://colab.research.google.com/github/nbeaudoin/LangChain-Experimentation-Zone/blob/main/Using_CSV_%26_Excel_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tutorial Source: https://www.youtube.com/watch?v=xQ3mZhw69bc&list=PL8motc6AQftk1Bs42EW45kwYbyJ4jOdiZ&index=16

In [5]:
!pip -q install openai langchain python-dotenv

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
from dotenv import load_dotenv

# Load variables from file
load_dotenv('/content/drive/MyDrive/Projects/keys/secrets.json')

# Use variables
openai_api = os.getenv('OPENAI_API_KEY')

# Dataset - Black Friday Sales

In [7]:
# Get data from Kaggle
!wget -q https://www.dropbox.com/s/8y6a1zloiscuo5d/black_friday_sales.zip
!unzip -q black_friday_sales.zip

In [9]:
import pandas as pd

df = pd.read_csv('/content/train.csv')

In [10]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


# CSV Agent

In [11]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

In [20]:
agent = create_csv_agent(OpenAI(temperature=0),
                         '/content/train.csv',
                         verbose=True)

In [17]:
agent.agent.llm_chain.prompt.template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df_head}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

In [21]:
agent.run("How many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)
Observation: 550068
Thought: I now know the final answer
Final Answer: There are 550068 rows in the dataframe.

> Finished chain.


'There are 550068 rows in the dataframe.'

In [22]:
agent.run("How many people are female?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of people who are female
Action: python_repl_ast
Action Input: df[df['Gender'] == 'F'].shape[0]
Observation: 135809
Thought: I now know the final answer
Final Answer: 135809

> Finished chain.


'135809'

In [23]:
agent.run("What percentage of people are female?")



> Entering new AgentExecutor chain...
Thought: I need to find the total number of people and the number of females
Action: python_repl_ast
Action Input: df['Gender'].value_counts(normalize=True)
Observation: M    0.753105
F    0.246895
Name: Gender, dtype: float64
Thought: I now know the final answer
Final Answer: 24.6895%

> Finished chain.


'24.6895%'

In [24]:
agent.run("How many people have stayed more than 3 years in the city?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to get the people who have stayed more than 3 years in the city.
Action: python_repl_ast
Action Input: df[df['Stay_In_Current_City_Years'] > '3'].shape[0]
Observation: 84726
Thought: I now know the final answer
Final Answer: 84726 people have stayed more than 3 years in the city.

> Finished chain.


'84726 people have stayed more than 3 years in the city.'

In [25]:
agent.run("How many people have stayed more than 3 years in the city and are female?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to find the people who meet the criteria
Action: python_repl_ast
Action Input: df[(df['Stay_In_Current_City_Years'] > '3') & (df['Gender'] == 'F')]
Observation:         User_ID Product_ID Gender    Age  Occupation City_Category  \
29      1000010  P00085942      F  36-45           1             B   
30      1000010  P00118742      F  36-45           1             B   
31      1000010  P00297942      F  36-45           1             B   
32      1000010  P00266842      F  36-45           1             B   
33      1000010  P00058342      F  36-45           1             B   
...         ...        ...    ...    ...         ...           ...   
549972  1005901  P00372445      F  36-45           7             C   
549981  1005911  P00372445      F  26-35          17             C   
550011  1005952  P00371644      F  46-50           1             C   
550065  1006036  P00375436      F  26-35          15       

'18,596'

In [26]:
agent.run("Are there more males than females?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of males and females
Action: python_repl_ast
Action Input: df['Gender'].value_counts()
Observation: M    414259
F    135809
Name: Gender, dtype: int64
Thought: There are more males than females
Final Answer: Yes, there are more males than females.

> Finished chain.


'Yes, there are more males than females.'